In [2]:
import pycephes
from scipy.special import bdtr as scipy_btdr

Let's time the SciPy Binomial Distribution function:

In [6]:
%timeit scipy_btdr(4, 6, 0.3)


1.75 µs ± 84.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Okay, running the SciPy version takes 1.75 microseconds. That's practically nothing, surely we can't beat that?

Hold my beer.

In [9]:
%timeit pycephes.bdtr(4, 6, 0.3)


475 ns ± 133 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Running the PyCephes version takes 308 nanoseconds, or 0.308 microseconds. An 82.5% speedup. 

Moreover, you cannot use SciPy's special functions in Numba-jitted functions running in nopython mode, which might introduce a bottleneck to your code. PyCephes functions, on the other hand, can...